In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
## This is a notebook for running the model that predicts ED and MCES20
## Big difference with previous models: We can't use cosine similarity to compute analog discovery
## I have to:
## 1) copy the mces20-ed model from vsc to local folder
## 2) Replace the computation of the cosine similary by the computation of the fully connected layer

In [ ]:
import os
os.chdir('/Users/sebas/projects/metabolomics')
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

In [ ]:
from simba.transformers.sklearn_model import SklearnModel
import gensim
from simba.load_data import LoadData
from simba.config import Config
from tqdm import tqdm
from simba.loader_saver import LoaderSaver
import itertools
import numpy as np
from scipy.stats import spearmanr
import dill
from simba.plotting import Plotting
from simba.load_data import LoadData
from simba.molecule_pairs_opt import MoleculePairsOpt
from sklearn.model_selection import train_test_split
from simba.train_utils import TrainUtils
from simba.preprocessor import Preprocessor
import pickle
import sys
from simba.config import Config
from simba.parser import Parser
from datetime import datetime
from simba.loader_saver import LoaderSaver
from simba.molecular_pairs_set import MolecularPairsSet
from scipy.stats import spearmanr
from simba.transformers.embedder import Embedder
from simba.transformers.encoder import Encoder
import matplotlib.pyplot as plt
from simba.transformers.CustomDatasetEncoder import CustomDatasetEncoder
from simba.transformers.load_data_encoder import LoadDataEncoder
from simba.transformers.load_data_unique import LoadDataUnique
from torch.utils.data import DataLoader
from simba.analog_discovery.cosine_similarity import CosineSimilarity
from rdkit import Chem
from simba.analog_discovery.analog_discovery import AnalogDiscovery
from simba.analog_discovery.mces import MCES
from simba.analog_discovery.ad_multitask import ADMultitask
from simba.analog_discovery.fc_layers_analog_discovery import FcLayerAnalogDiscovery
import copy
from simba.ordinal_classification.embedder_multitask import EmbedderMultitask
from simba.plotting import Plotting
import simba
from simba.tanimoto import Tanimoto
import sys
sys.modules['src']=simba

## params

In [ ]:
MULTITASKING=True

In [ ]:
data_folder= '/Users/sebas/projects/data/'
janssen_path = data_folder + 'drug_plus.mgf'
output_janssen_file= data_folder + 'all_spectrums_janssen.pkl'
#dataset_path= data_folder +'merged_gnps_nist_20240311_unique_smiles_1_million.pkl'
#dataset_path= data_folder +'merged_gnps_nist_20240319_unique_smiles_1_million_v2_no_identity.pkl'
dataset_path= data_folder +'edit_distance_neurips_nist_exhaustive.pkl'


#model_path = data_folder + 'best_model_generated_data_20250126.ckpt'  
#model_path = data_folder + 'best_model_generated_data_20250130.ckpt'  ## BEST RESULTS UNTIL NOW
#model_path = data_folder + 'best_model_20250411_penalty_matrix.ckpt'
#model_path = data_folder + 'best_model_20250422_only_massspecgym.ckpt'
#model_path = data_folder + 'best_model_generated_data_20250206_resampling.ckpt'  
model_path = data_folder + 'best_model_20250424_fingerprint.ckpt'
casmi_file= data_folder + 'processed_massformer/spec_df.pkl' 

In [ ]:
#similarity_name= 'simba_EDIT_DISTANCE_WITH_CONFIDENCE'
#similarity_name= 'simba_EDIT_DISTANCE'
#similarity_name= 'simba_EDIT_DISTANCE_MCES20_NEURIPS_precursor_randomized'
#similarity_name= 'simba_generated_data_20250130'
#similarity_name= 'simba_20250411_penalty_matrix'
#similarity_name= "simba_generated_data_20250130_replicated"
similarity_name= 'simba_20250424_fingerprint'
#similarity_name= 'simba_20250422_only_massspecgym'
#similarity_name= 'simba_EDIT_DISTANCE_MCES20_NEURIPS_2METRICS'
#similarity_name= 'simba_EDIT_DISTANCE_PRETRAINING_WITHOUT_CONFIDENCE'

In [ ]:
config=Config()

In [ ]:
config.D_MODEL=256
#config.D_MODEL=512
config.N_LAYERS=5
config.TRANSFORMER_CONTEXT=100
config.USE_FINGERPRINT=1

In [ ]:
#if model_path.endswith('best_model_mces20_ed_optimized_20241017.ckpt'):
config.use_cosine_distance=True
#else:
#    config.use_cosine_distance=False

In [ ]:
model_path

In [ ]:
config.use_cosine_distance

In [ ]:
USE_CASMI=True
USE_MCES_RAW=False

In [ ]:
USE_MS2DEEPSCORE =False
if USE_MS2DEEPSCORE:
    similarity_name= 'simba_SIMBA_ENSEMBLED_MS2DEEPSCORE_SPEC2VEC'

In [ ]:
data_folder= '/Users/sebas/projects/data/'
model_ms2d_file = data_folder + 'ms2deepscore_positive_10k_1000_1000_1000_500.hdf5'
model_spec2vec_file = data_folder + 'spec2vec_AllPositive_ratio05_filtered_201101_iter_15.model'

In [ ]:
from ms2deepscore import MS2DeepScore
from ms2deepscore.models import load_model
import tensorflow as tf
from matchms import calculate_scores
from simba.matchms_utils import MatchmsUtils
from spec2vec import Spec2Vec

class Ms2wrapper:
    def __init__(self, model_ms2d_file):
        self.model_ms2d = load_model(model_ms2d_file)
        self.similarity_model= MS2DeepScore(self.model_ms2d)
        
    def get_predictions_ms2deepscore(self,s1, s2):
        results_ms2_all=[]
        s_query= MatchmsUtils.from_su_to_matchms(s1)
        s_ref= MatchmsUtils.from_su_to_matchms(s2)
        with tf.device('/device:CPU:0'): #execute on cpu
                results_ms2 = calculate_scores([s_query], [s_ref], self.similarity_model)
        return results_ms2.to_array()

    def compute_similarity_matrix(self,specs_su_1, specs_su_2):
        results_ms2_all=[]
        s_query= [MatchmsUtils.from_su_to_matchms(s1) for s1 in specs_su_1]
        s_ref=  [MatchmsUtils.from_su_to_matchms(s2) for s2 in specs_su_2]
        with tf.device('/device:CPU:0'): #execute on cpu
                results_ms2 = calculate_scores(s_query, s_ref, self.similarity_model)
        return results_ms2.to_array()

class Spec2vec_wrapper:
    def __init__(self, model_ms2d_file):
        self.model = gensim.models.Word2Vec.load(model_spec2vec_file)
        self.similarity_model= Spec2Vec(
                model=self.model, intensity_weighting_power=0.5, allowed_missing_percentage=100.0
            )
    def compute_similarity_matrix(self,specs_su_1, specs_su_2):
        s_query= [MatchmsUtils.from_su_to_matchms(s1) for s1 in specs_su_1]
        s_ref=  [MatchmsUtils.from_su_to_matchms(s2) for s2 in specs_su_2]
        with tf.device('/device:CPU:0'): #execute on cpu
                results_spec2vec_all = calculate_scores(s_query, s_ref, self.similarity_model)
        return results_spec2vec_all.to_array()

In [ ]:
#ms2_object=Spec2vec_wrapper(model_spec2vec_file)
#ms2_object=Ms2wrapper(model_ms2d_file)

In [ ]:
def get_similarities_combined(sim1, sim2_softmax, threshold_ed5=0.9):
    '''
    return a similarity proxy based on sim1 and sim2
    '''
    #return sim1*(sim2>0)
    probability_ed5= sim2_softmax[:,:,0]
    return sim1*(~((probability_ed5>threshold_ed5)  & (sim1>0.50))  ) ## remove bad predictions

## open janssen data

In [ ]:
if USE_CASMI:
    all_spectrums_janssen_original=LoadData.get_all_spectrums_casmi(
        casmi_file,
        config=config,
    )
else:
    loader_saver = LoaderSaver(
            block_size=100,
            pickle_nist_path='',
            pickle_gnps_path='',
            pickle_janssen_path=output_janssen_file,
        )
    all_spectrums_janssen_original = loader_saver.get_all_spectrums(
                janssen_path,
                100000000,
                use_tqdm=True,
                use_nist=False,
                config=config,
                use_janssen=True,
            )

In [ ]:
ls projects/data/processed_massformer

In [ ]:
all_spectrums_janssen_processed= [copy.deepcopy(s) for s in all_spectrums_janssen_original]
pp=Preprocessor()
### remove extra peaks in janssen
all_spectrums_janssen_processed = [pp.preprocess_spectrum(
            s,
            fragment_tol_mass=10,
            fragment_tol_mode="ppm",
            min_intensity=0.01,
            max_num_peaks=1000,
            scale_intensity=None,
        ) for s in all_spectrums_janssen_processed]


all_spectrums_janssen = [s_original for s_original, s_processed in zip(all_spectrums_janssen_original,all_spectrums_janssen_processed) if len(s_processed.mz)>=6]

## Filter only one spectra?

## open ref data

In [ ]:
with open(dataset_path, 'rb') as file:
            dataset = dill.load(file)

In [ ]:
dataset['molecule_pairs_test']

In [ ]:
all_spectrums_reference_original= dataset['molecule_pairs_train'].spectrums_original +\
                    dataset['molecule_pairs_val'].spectrums_original + \
                        dataset['molecule_pairs_test'].spectrums_original


In [ ]:
## Lets make an experiment
EXPERIMENT=False
if EXPERIMENT:
    from simba.config import Config
    from simba.simba.simba import Simba
    from simba.simba.preprocessing_simba import PreprocessingSimba
    mgf_file = '/Users/sebas/projects/data/ALL_GNPS_NO_PROPOGATED_wb.mgf'
    all_spectrums=PreprocessingSimba.load_spectra(mgf_file, config)
    all_spectrums_janssen_original = copy.deepcopy(all_spectrums[0:10])
    all_spectrums_reference_original = copy.deepcopy(all_spectrums[0:20])


In [ ]:
import copy


In [ ]:
all_spectrums_reference_processed= [copy.deepcopy(s) for s in all_spectrums_reference_original]
all_spectrums_reference_processed = [pp.preprocess_spectrum(
            s,
            fragment_tol_mass=10,
            fragment_tol_mode="ppm",
            min_intensity=0.01,
            max_num_peaks=1000,
            scale_intensity='root',
        ) for s in all_spectrums_reference_processed]
    
all_spectrums_reference = [s_original for s_original, s_processed in zip(all_spectrums_reference_original,all_spectrums_reference_processed) if len(s_processed.mz)>=6]

In [ ]:
## extra filtering to discard bad spectra in the reference dataset
all_spectrums_reference_processed_2= [copy.deepcopy(s) for s in all_spectrums_reference_original]
all_spectrums_reference_processed_2 = [pp.preprocess_spectrum(
            s,
            fragment_tol_mass=10,
            fragment_tol_mode="ppm",
            min_intensity=0.1,
            max_num_peaks=100,
            scale_intensity=None,
        ) for s in all_spectrums_reference_processed_2]
    
more_informative_spectrums = [1 if len(s.mz)>2 else 0 for s in all_spectrums_reference_processed_2]

## histogram of peaks

In [ ]:
len_z_ref= np.array([len(s.mz) for s in all_spectrums_reference_processed])


In [ ]:
len(len_z_ref)

In [ ]:
min(len_z_ref)

In [ ]:
mz_ref= np.array([s.precursor_mz for s in all_spectrums_reference_processed] , )
_=plt.hist(mz_ref[np.random.randint(0,len(mz_ref), 10000)], bins=30, alpha=0.5,label='ref' , density=True)

In [ ]:
mz_ref= np.array([s.precursor_mz for s in all_spectrums_janssen_processed] , )
_=plt.hist(mz_ref[np.random.randint(0,len(mz_ref), 10000)], bins=30, alpha=0.5,label='ref' , density=True)


In [ ]:
len_z_ref= np.array([len(s.mz) for s in all_spectrums_reference] , )
_=plt.hist(len_z_ref[np.random.randint(0,len(len_z_ref), 1000)], bins=30, alpha=0.5, label='ref original', density=True)
len_z_ref= np.array([len(s.mz) for s in all_spectrums_reference_processed], )
_=plt.hist(len_z_ref[np.random.randint(0,len(len_z_ref), 10000)], bins=30, alpha=0.5,label='ref processed' , density=True)
plt.legend()

In [ ]:
len_z_query= np.array([len(s.mz) for s in all_spectrums_janssen_processed] , )
_=plt.hist(len_z_query[np.random.randint(0,len(len_z_query), 1000)], bins=30, alpha=0.5, label='query processed', density=True)
len_z_ref= np.array([len(s.mz) for s in all_spectrums_reference_processed], )
_=plt.hist(len_z_ref[np.random.randint(0,len(len_z_ref), 10000)], bins=30, alpha=0.5,label='ref processed' , density=True)
plt.legend()

In [ ]:
import random
random.random()

In [ ]:
min(len_z_ref)

In [ ]:
_=plt.hist(len_z_ref[np.random.randint(0,len(len_z_ref), 1000)], bins=30)

In [ ]:
#proportion of spectra that is lower than 20
len_z_ref[len_z_ref<40].shape[0]/len_z_ref.shape[0]

## Find those instances that are not in reference

In [ ]:
canon_smiles_reference = [Chem.CanonSmiles(s.smiles) for s in all_spectrums_reference]
canon_smiles_janssen =   [Chem.CanonSmiles(s.smiles) for s in all_spectrums_janssen]
janssen_indexes_in_ref= [i for i,s in enumerate(canon_smiles_janssen) if s in canon_smiles_reference]
janssen_indexes_not_in_ref = [i for i,s in enumerate(canon_smiles_janssen) if s not in canon_smiles_reference]

In [ ]:
len(janssen_indexes_in_ref),len(janssen_indexes_not_in_ref)

In [ ]:
all_spectrums_janssen = [all_spectrums_janssen[index] for index in janssen_indexes_not_in_ref]

In [ ]:
all_spectrums_janssen[0].mz

In [ ]:
#all_spectrums_janssen = copy.deepcopy(all_spectrums[0:10])
#all_spectrums_reference = copy.deepcopy(all_spectrums[0:20])

## load encoder

In [ ]:
encoder= Encoder(model_path, D_MODEL=int(config.D_MODEL),
                 N_LAYERS=int(config.N_LAYERS), 
                 multitasking=MULTITASKING, 
                 config=config)

## data loader

In [ ]:
dataset_janssen= LoadDataEncoder.from_spectrums_to_dataset(all_spectrums_janssen, max_num_peaks=int(config.TRANSFORMER_CONTEXT),)
#dataset_janssen= LoadDataEncoder.from_spectrums_to_dataset(all_spectrums_janssen, max_num_peaks=20,)
dataloader_janssen= DataLoader(dataset_janssen, batch_size=config.BATCH_SIZE,  num_workers=0)

In [ ]:
dataset_reference= LoadDataEncoder.from_spectrums_to_dataset(all_spectrums_reference,  max_num_peaks=int(config.TRANSFORMER_CONTEXT),)
#dataset_reference= LoadDataEncoder.from_spectrums_to_dataset(all_spectrums_reference, max_num_peaks=20,)
dataloader_reference= DataLoader(dataset_reference, batch_size=config.BATCH_SIZE,  num_workers=0)

In [ ]:
#dataset_nist= LoadDataEncoder.from_spectrums_to_dataset(all_spectrums_nist)
#dataloader_nist= DataLoader(dataset_nist, batch_size=config.BATCH_SIZE,  num_workers=0)

In [ ]:
#dataset_gnps= LoadDataEncoder.from_spectrums_to_dataset(all_spectrums_gnps)
#dataloader_gnps= DataLoader(dataset_gnps, batch_size=config.BATCH_SIZE,  num_workers=0)

In [ ]:
for batch in dataloader_reference:
    batch_example=batch
    print(batch['intensity'])
    plt.plot(batch['intensity'][10])
    break

## forward

In [ ]:
embeddings_janssen= encoder.get_embeddings(dataloader_janssen)

In [ ]:
embeddings_reference = encoder.get_embeddings(dataloader_reference)

In [ ]:
#embeddings_nist = encoder.get_embeddings(dataloader_nist)

In [ ]:
#embeddings_gnps = encoder.get_embeddings(dataloader_gnps)

## compute similarities simba

In [ ]:
config.use_cosine_distance

In [ ]:
config.EDIT_DISTANCE_USE_GUMBEL

In [ ]:
if config.USE_FINGERPRINT:
    fingerprints_0 = np.array([Tanimoto.compute_fingerprint(s.params['smiles']) for s in all_spectrums_reference])
else:
    fingerprints_0=None

In [ ]:
embeddings_janssen.shape

In [ ]:
embeddings_reference.shape

In [ ]:

## In this context we are treating similarities2 as edit distance
#if config.use_cosine_distance:
#    #similarities1= CosineSimilarity.batch_cosine_similarity_combinations(embeddings_janssen, embeddings_reference)
#else:
similarities2, similarities1=FcLayerAnalogDiscovery.compute_all_combinations(model_path, 
                                                                                 embeddings_janssen, 
                                                                                 embeddings_reference, 
                                                                                 config,
                                                                                    fingerprints_0=fingerprints_0, 
                                                                                     fingerprint_index=1)
## postprocessing similarities1
## we can't get similarities higher than 1.0
#similarities1[similarities1>1]=1

In [ ]:
similarities2[0,0]

In [ ]:
similarities1[0,0]

In [ ]:
similarities1.shape

In [ ]:
similarities2.shape

In [ ]:
similarities2_norm = np.argmax(similarities2, axis=2)/5

In [ ]:
similarities2_norm.shape

In [ ]:
plt.hist(similarities1.reshape(-1)[0:1000])

In [ ]:
(similarities2_norm>0)

In [ ]:
similarities1.shape[0]*similarities1.shape[1]

In [ ]:
similarities1[(similarities1>0.9)].shape

In [ ]:
# how many pairs have high similarity1 
similarities1[(similarities1>0.9)&(similarities2_norm < 0.5)].shape

In [ ]:
similarities1[(similarities1>0.9)].shape

In [ ]:
# how many pairs have high similarity1 and low similarity 2?
similarities1[similarities1>=1].shape

In [ ]:
similarities2_norm

In [ ]:
# how many pairs have high similarity1  and low similarity 2?
similarities1[(similarities1>=0.75)].shape

In [ ]:
# how many pairs have high similarity1  and low similarity 2?
similarities1[(similarities1>0.75)&(similarities2_norm==0)].shape

## Combine both metrics

In [ ]:
def softmax(x):
    # Subtract the maximum value in the last dimension to stabilize the computation
    e_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return e_x / np.sum(e_x, axis=-1, keepdims=True)
similarities2_softmax= softmax(similarities2)

In [ ]:
ranking_1 = np.zeros(similarities1.shape)
for row_index, row_similarities in enumerate(similarities1):
    indexes_sorted= np.argsort(row_similarities)[::-1]
    ranking_1[row_index] = np.argsort(indexes_sorted)

#normalize
ranking_1 = 1- ranking_1/np.max(ranking_1)

In [ ]:
similarities2_integer = similarities2_norm*5

In [ ]:
similarities1_integer = np.array((similarities1*40))
similarities1_integer= np.round(similarities1_integer)

In [ ]:
import numpy as np

def compute_ranking(similarities1, similarities1_integer, similarities2_integer, max_value_2_int=5):
    # Preallocate the ranking array with the same shape as similarities1.
    ranking_total = np.zeros(similarities1.shape, dtype=int)
    
    # Process each row (or each set of values) individually.
    for row_index, (row_sim, row_int, row_int2) in enumerate(zip(similarities1, similarities1_integer, similarities2_integer)):
        # Use lexsort with a composite key:
        #   - Primary: similarities1_integer (ascending)
        #   - Secondary: similarities2_integer (ascending)
        #   - Tertiary: similarities1 (descending, so use -row_sim)
        #
        # Note: np.lexsort uses the last key as the primary key.
        sorted_indices = np.lexsort( ( row_sim, row_int2, row_int ) )
        
        # Now assign ranking values based on sorted order.
        # Here the best (first in sorted_indices) gets rank 0,
        # the next gets rank 1, etc.
        ranking = np.empty_like(sorted_indices)
        ranking[sorted_indices] = np.arange(len(row_sim))
        
        # Store the ranking for this row.
        ranking_total[row_index] = ranking

    #normalizing
    ranking_total =ranking_total/ranking_total.shape[1]
    return ranking_total

In [ ]:
ranking_total= compute_ranking(similarities1, similarities1_integer, similarities2_integer)

In [ ]:
np.max(ranking_1)

In [ ]:
## the first 100 positions
proportion1000= 1000/len(all_spectrums_reference)

In [ ]:
proportion1000

In [ ]:
np.min(ranking_1)

In [ ]:
# based on edit distance=0 output
#similarities2_ed5_complementary=1-similarities2_softmax[:,:,0]
#ranking_2 = np.zeros(similarities2_ed5_complementary.shape)
#for row_index, row_similarities in enumerate(similarities2_ed5_complementary):
#    indexes_sorted= np.argsort(row_similarities)[::-1]
#    ranking_2[row_index] = np.argsort(indexes_sorted)
#ranking_2 = 1- ranking_2/np.max(ranking_2)

import pandas as pd
# based on similarity norm
def normalize_ranks(values):
    # Convert values to a numpy array for processing
    original_shape=values.shape
    values=np.reshape(values, (-1))
    values = np.array(values)
    
    # Get ranks using pandas (average ranking for ties)
    ranks = pd.Series(values).rank(method='average').to_numpy()
    
    # Normalize ranks to [0, 1]
    normalized_ranks = (ranks - 1) / (len(values) - 1)

    normalized_ranks= np.reshape(normalized_ranks, original_shape)
    return normalized_ranks

# Get normalized ranks
ranking_2 = normalize_ranks(similarities2_norm)



In [ ]:
similarities1.shape

In [ ]:
similarities1[similarities1>0.9].shape

In [ ]:
similarities1[(similarities2_norm>0) &(similarities1>0.90)].shape

In [ ]:
## combine both metrics
#similarities= similarities1
similarities= ranking_total
#similarities= similarities1*(similarities2_norm>0)
#similarities= ((ranking_1+ranking_2)/2)
#similarities = ((ranking_1+ranking_2)/2) *(ranking_1> (1-proportion1000))*(ranking_2>(1-proportion1000))
#similarities = similarities1 *(ranking_1> (0.95))*(ranking_2>(0.95))  ### this soft filtering improves a little bit the results
#similarities= similarities1*(similarities2_norm>0)
#similarities = similarities1*(similarities2_softmax[:,:,0]<0.1)

In [ ]:
similarities1[similarities1>0.95].shape

In [ ]:
similarities1[(similarities1>0.95) & (similarities2_norm>0.0)].shape

In [ ]:
similarities1[(similarities1>0.95) & (similarities2_softmax[:,:,0]<0.10)].shape

In [ ]:
#similarities= similarities1*(similarities1<0.98) + similarities1*((similarities1>=0.98) & (similarities2_softmax[:,:,0]<0.01))

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(similarities[:, 0:100], cmap='hot', interpolation='nearest')
#plt.colorbar()  # Add colorbar to show scale
plt.title('Heatmap of Matrix')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.show()

In [ ]:
all_spectrums_reference[3].params

In [ ]:
smiles=[s.params['smiles'] for s in all_spectrums_reference]

## Check distribution of outputs

In [ ]:
N=1000
random_j= np.random.randint(0,similarities.shape[0],N)
random_r=np.random.randint(0,similarities.shape[1],N)

In [ ]:
plt.hist(similarities1[random_j,random_r], bins=20)

In [ ]:
plt.hist(similarities2_norm[random_j,random_r], bins=20)

In [ ]:
plt.hist(similarities[random_j,random_r], bins=20)

## Retrieval of best candidates

In [ ]:
similarities.shape

In [ ]:
spectrums_k_retrieved, tanimoto_k_retrieved, sim_k_retrieved, arg_max_k10 =\
                AnalogDiscovery.get_k_candidates(similarities, all_spectrums_reference, 
                                                 all_spectrums_janssen,k=10, return_arg_max_k10=True)

In [ ]:
from simba.analog_discovery.mces import MCES
if USE_MCES_RAW:
    mces_k_retrieved= []
    for s_group, spec_janssen in zip(spectrums_k_retrieved,all_spectrums_janssen):
        smiles_0 = [s.params['smiles'] for s in s_group]
        smiles_1 = [spec_janssen.params['smiles']]*len(s_group)
        df = MCES.compute_mces_list_smiles(smiles_0, smiles_1, threshold_mces=20)
        mces_k_retrieved.append(df['mces'].values)
else:
    mces_k_retrieved = [[MCES.calculate_mces_sim(s.params['smiles'], spec_janssen.params['smiles']) for s in s_group] for s_group, spec_janssen in zip(spectrums_k_retrieved,all_spectrums_janssen)]

In [ ]:
#mces_k_retrieved = [[MCES.calculate_mces_sim(s.params['smiles'], spec_janssen.params['smiles']) for s in s_group] for s_group, spec_janssen in zip(spectrums_k_retrieved,all_spectrums_janssen)]

In [ ]:
#best_indexes = [np.argmax(tani_group[0:10]) if len(tani_group)>=1 else None for tani_group in tanimoto_k_retrieved ]
#best_indexes = [np.argmax([m for m in mces_group if m is not None][0:10]) if len(mces_group)>=1 else None for mces_group in tanimoto_k_retrieved ]
#best_indexes = [np.argmax([m for m in mces_group if m is not None]) if len(mces_group)>=0 else 0 for mces_group in tanimoto_k_retrieved ]
#best_indexes = [np.argmax([m for m in mces_group if m is not None]) if len(mces_group)>=0 else 0 for mces_group in mces_k_retrieved ]
if USE_MCES_RAW:
    best_indexes = [np.argmin([m for m in mces_group if m is not None]) if len(mces_group)>=0 else 0 for mces_group in mces_k_retrieved ]
else: #if using normalized similarity
    best_indexes = [np.argmax([m for m in mces_group if m is not None]) if len(mces_group)>=0 else 0 for mces_group in mces_k_retrieved ]

In [ ]:
# get the best tanimotos
spectrums_retrieved = [spectrums_k_retrieved[index_spectrum][best_index]  for index_spectrum, best_index in enumerate(best_indexes) if best_index is not None]
tanimoto_retrieved = [tanimoto_k_retrieved[index_spectrum][best_index]  for index_spectrum, best_index in enumerate(best_indexes) if best_index is not None]
mces_sims = [mces_k_retrieved[index_spectrum][best_index]  for index_spectrum, best_index in enumerate(best_indexes) if best_index is not None]

max_sim = [sim_k_retrieved[index_spectrum][best_index]  for index_spectrum, best_index in enumerate(best_indexes) if best_index is not None]

In [ ]:
# retrieve of the edit distance result
ed_pred_retrieved= [similarities2_softmax[index_spectrum ,arg_max_k10[index_spectrum]]     for index_spectrum, best_index in enumerate(best_indexes) if best_index is not None]
ed_sim = [similarities2_softmax[index_spectrum ,arg_max_k10[index_spectrum][best_index]]     for index_spectrum, best_index in enumerate(best_indexes) if best_index is not None]


In [ ]:
smiles_retrieved= [s.smiles for s in spectrums_retrieved]

In [ ]:
_=plt.hist(max_sim, color='r', bins=10)
plt.xlabel('maximum SIMBA predicted similarity found')
plt.ylabel('freq')
plt.grid()

In [ ]:
plt.hist(tanimoto_retrieved,density=True, bins=10)
plt.grid()
plt.xlabel('tanimoto similarity with reference spectra')
plt.ylabel('frequency')
plt.ylim([0,3.5])

In [ ]:
plt.boxplot(tanimoto_retrieved)
plt.grid()

In [ ]:
all_smiles_janssen = [s.params['smiles'] for s in all_spectrums_janssen]

In [ ]:
mces_sims=[]
for s0,s1 in tqdm(zip(all_smiles_janssen, smiles_retrieved)):
    similarity= MCES.calculate_mces_sim(s0, s1)
    mces_sims.append(similarity)

In [ ]:


# best indexes taking into account ED threshold
valid_ed_predictions = [ [p if p[0]<0.1 else  None for p in p_list] for p_list in ed_pred_retrieved]
best_indexes_ed= [[s if v is not None else 0 for s,v in zip(s_list,v_list) ] for v_list,s_list in zip(valid_ed_predictions, sim_k_retrieved)]
best_indexes_ed = [np.argmax(b) if np.max(b)>0 else None for b in best_indexes_ed]
smiles_retrieved_ed = [spectrums_k_retrieved[index_spectrum][best_index].params['smiles']  if best_index is not None else None for index_spectrum, best_index in enumerate(best_indexes_ed) ]
mces_sims_ed=[]
for s0,s1 in tqdm(zip(all_smiles_janssen, smiles_retrieved_ed)):
    if s1 is not None:
        similarity= MCES.calculate_mces_sim(s0, s1)
        mces_sims_ed.append(similarity)

In [ ]:
mces_without_confidence= [m for m in mces_sims if m is not None]
mces_with_confidence= [m for m,ed in zip(mces_sims, ed_sim) if ((m is not None)and (ed[0]<0.01))]

plt.figure(figsize=(7,3))
plt.title('Improvement by confidence metric (edit distance)')
plt.boxplot([mces_without_confidence,mces_with_confidence], labels= ['No confidence score', 'Confidence score'])
#plt.boxplot(mces_with_confidence)
#plt.boxplot([m for m in mces_retrieved if m is not None])
plt.grid()
print(f'Proportion of confident predictions: {len(mces_with_confidence)/len(mces_without_confidence)}')
print(f'Median of confident predictions: {np.median(mces_with_confidence)}')
#plt.ylim([0,1.1])

In [ ]:
plt.boxplot([m for m in mces_sims if m is not None])
#plt.boxplot([m for m in mces_retrieved if m is not None])
plt.grid()
#plt.ylim([0,1.1])

In [ ]:
np.median([m for m in mces_sims if m is not None])

## Sanity check

In [ ]:
from simba.ordinal_classification.embedder_multitask import EmbedderMultitask

In [ ]:
target_janssen_index= 0
target_ref_index=np.argsort(similarities[target_janssen_index])[::-1][0]


In [ ]:
spec_ref=    all_spectrums_reference[target_ref_index]
spec_janssen=all_spectrums_janssen[target_janssen_index]

In [ ]:
# query molecule
Chem.MolFromSmiles(spec_janssen.params['smiles'])

In [ ]:
# reference molecule
Chem.MolFromSmiles(spec_ref.params['smiles'])

In [ ]:
smiles_ref= spec_ref.params['smiles']
smiles_janssen= spec_janssen.params['smiles']

In [ ]:
from simba.tanimoto import Tanimoto

In [ ]:
Tanimoto.compute_tanimoto_from_smiles(smiles_ref,smiles_janssen)

In [ ]:
# mces 20
mces_norm = similarities1[target_janssen_index, target_ref_index]
print(f'mces distance: {config.MCES20_MAX_VALUE*(1-mces_norm)}')

In [ ]:
# edit distance
ed_vector=similarities2[target_janssen_index, target_ref_index]
print(ed_vector)
print(f'Edit distance found: {5-np.argmax(ed_vector)} modifications')

In [ ]:
similarities1[similarities1>0.99].shape

In [ ]:
ed_total=np.argmax(similarities2, axis=2)
ed_total=ed_total.reshape(-1)
ed_total[ed_total==5].shape

In [ ]:
ed_total[ed_total==5].shape[0]/(ed_total.shape[0])

In [ ]:
similarities1[similarities1>0.99].shape[0]/(similarities1.shape[0]*similarities1.shape[1])

## Let's make the ROC curve

In [ ]:
## 1) for the roc curve we have to get a set of true MCES similarities 
## get the MCES predicted by the model
## compute the num bonds for each molecule of the pair
## compute similarity based on the distance computed

In [ ]:
number_examples=10000

In [ ]:
random_indexes_0= np.random.randint(0,len(all_spectrums_janssen),number_examples)

In [ ]:
random_indexes_1=np.random.randint(0,len(all_spectrums_reference),number_examples)

In [ ]:
# get the spectra
spectrums_random_0 = [all_spectrums_janssen[index] for index in random_indexes_0]
spectrums_random_1 = [all_spectrums_reference[index] for index in random_indexes_1]

In [ ]:

def softmax(x):
    # Subtract the maximum value in the last dimension to stabilize the computation
    e_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return e_x / np.sum(e_x, axis=-1, keepdims=True)

similarities2_softmax= softmax(similarities2)

In [ ]:
similarities2_softmax[0,0]

In [ ]:
## compute ground truth
mces20_groundtruth = []
for s0,s1 in tqdm(zip(spectrums_random_0, spectrums_random_1)):
    similarity= MCES.calculate_mces_sim(s0.params['smiles'], s1.params['smiles'],similarity_threshold=0.7)
    mces20_groundtruth.append(similarity)

In [ ]:
num_bonds_0 = [Chem.MolFromSmiles(s.params['smiles']).GetNumBonds() for s in spectrums_random_0]
num_bonds_1 = [Chem.MolFromSmiles(s.params['smiles']).GetNumBonds() for s in spectrums_random_1]

In [ ]:
threshold=0.7
thresholds_ed=[1, 0.1]
colors= ['b','r','g']

In [ ]:
## try different thresholds
y_true_list =[]
y_scores_list=[]
coverage_list=[]
for threshold_ed5, color in zip(thresholds_ed, colors):
    # use the edit distance
    similarities_roc= get_similarities_combined(similarities1, similarities2_softmax,threshold_ed5)

    mces20 = 40*(1-similarities_roc)

    # sample based on ed
    mces20_sampled = [mces20[index_0, index_1] for index_0, index_1 in zip(random_indexes_0, random_indexes_1)]

    predicted_similarity =  [((n0+n1)-m20)/(n0+n1)  for n0,n1,m20 in zip(num_bonds_0,num_bonds_1, mces20_sampled)]

    
    valid_indexes = [i for i,m in enumerate(mces20_sampled) if m!=40 ]

    coverage=(len(valid_indexes)/len(mces20_sampled))
    coverage = coverage if coverage<0.98 else 1
    coverage=100*coverage #percentage

    mces20_groundtruth=np.array(mces20_groundtruth)
    predicted_similarity=np.array(predicted_similarity)

    y_true = [m if m is not None else 0 for m in mces20_groundtruth[valid_indexes] ]
    y_true=[1 if m>threshold else 0 for m in y_true]
    y_scores= predicted_similarity[valid_indexes]

    y_true_list.append(y_true)
    y_scores_list.append(y_scores)
    coverage_list.append(coverage)
    #Plotting.plot_roc_curve(
    #    y_true,
    #    y_scores,
    #    label=f'coverage: {coverage:.2f}%',
    #    color=color)
    #plt.grid()

In [ ]:
Plotting.plot_n_roc_curves(y_true_list, y_scores_list, labels=[f'coverage: {coverage:.2f}%' for coverage in coverage_list],
        colors=['b','r','g'],
        linestyles=['-','-','-'],
            figsize=(6,6))

In [ ]:
len(mces20_sampled)

In [ ]:
mces20_groundtruth=np.array(mces20_groundtruth)
predicted_similarity=np.array(predicted_similarity)

In [ ]:
plt.scatter(mces20_groundtruth[valid_indexes], predicted_similarity[valid_indexes], alpha=0.1)
plt.grid()
plt.ylabel('mces normalized predicted')
plt.xlabel('mces normalized ground truth')

In [ ]:
## compute roc curve


In [ ]:
results ={ 'smiles_janssen' : [s.params['smiles'] for s in all_spectrums_janssen], 
        'smiles_reference' : [s.params['smiles'] for s in all_spectrums_reference], 
        'mces_retrieved':mces_sims,
          'similarities':similarities,
          'similarities1':similarities1,
          'similarities2':similarities2,
         }

In [ ]:
with open('./notebooks/discovery_search/results/'+similarity_name + '_results_analog_discovery_unknwon_compounds.pkl', 'wb') as f:
    dill.dump(results, f)

## Use of raw mces

In [ ]:
# let's use myopic

In [ ]:
from simba.mces.mces_computation import MCES

In [ ]:
#mces_k_retrieved = [[MCES.calculate_mces_sim(s.params['smiles'], spec_janssen.params['smiles']) for s in s_group] for s_group, spec_janssen in zip(spectrums_k_retrieved,all_spectrums_janssen)]

In [ ]:
#mces_retrieved_raw= []
#for s_group, spec_janssen in zip(spectrums_k_retrieved,all_spectrums_janssen):
#    smiles_0 = [s.params['smiles'] for s in s_group]
#    smiles_1 = [spec_janssen.params['smiles']]*len(s_group)
#    df = MCES.compute_mces_list_smiles(smiles_0, smiles_1, threshold_mces=20)
#    mces_retrieved_raw.append(df['mces'].values)

## Analysis of one example

In [ ]:
target_index=58

In [ ]:
target_query=all_spectrums_janssen[target_index]

In [ ]:
smiles_query= target_query.params['smiles']

In [ ]:
Chem.MolFromSmiles(smiles_query)

In [ ]:
len(target_query.mz)

In [ ]:
# load tanimotos
tanimoto_results_path = 'tanimoto_similarity_casmi_NEURIPS'
format_file_unknown = "./notebooks/discovery_search/results/{}_results_analog_discovery_unknwon_compounds.pkl"
with open(format_file_unknown.format(tanimoto_results_path), 'rb') as f:
                golden_truth= dill.load(f)['similarities']

In [ ]:
## What is the best match based on tanimoto?
best_golden_truth_index=np.argsort(golden_truth[target_index])[::-1][0]

In [ ]:
max_value=np.sort(golden_truth[target_index])[-1]
best_golden_truth_list = np.where(golden_truth[target_index]==max_value)[0]

In [ ]:
best_golden_truth_list

In [ ]:
Chem.MolFromSmiles(all_spectrums_reference[best_golden_truth_index].params['smiles'])

In [ ]:
len(all_spectrums_reference[best_golden_truth_index].mz)

In [ ]:
len(target_query.mz)

In [ ]:
similarities1.shape

## Define the desired ranking

In [ ]:
rank=0
highest_prediction_index=np.argsort(similarities[target_index])[::-1][rank]

In [ ]:
Chem.MolFromSmiles(all_spectrums_reference[highest_prediction_index].params['smiles'])

In [ ]:
len(all_spectrums_reference_processed[highest_prediction_index].mz)

In [ ]:
import spectrum_utils.plot as sup

## there is a peak disappearing after 300 for the query
#preprocess the spectrum and lets see how it looks like
import copy
target_query_copy= copy.deepcopy(target_query)
ref_copy= copy.deepcopy(all_spectrums_reference[highest_prediction_index])
pp=Preprocessor()
target_query_copy=pp.preprocess_spectrum(spectrum=target_query_copy)
ref_copy=pp.preprocess_spectrum(spectrum=ref_copy)
sup.mirror(target_query_copy, ref_copy, )

In [ ]:
import matplotlib.pyplot as plt

# Example plot for the query
query_batch = dataloader_janssen.dataset[target_index]
plt.scatter(query_batch['mz'], query_batch['intensity'], label='query')

# Add vertical lines for the query
plt.vlines(query_batch['mz'], 0, query_batch['intensity'], colors='blue', linestyles='--', alpha=0.7)

# Example plot for the reference
#ref_batch = dataloader_reference.dataset[best_golden_truth_index]
#highest_prediction_index
ref_batch = dataloader_reference.dataset[highest_prediction_index]
plt.scatter(ref_batch['mz'], ref_batch['intensity'], label='ref')

# Add vertical lines for the reference
plt.vlines(ref_batch['mz'], 0, ref_batch['intensity'], colors='orange', linestyles='--', alpha=0.7)

# Add legend and show the plot
plt.legend()
plt.xlabel('mz')
plt.ylabel('Intensity')
plt.title('Query vs Reference with Vertical Lines')
plt.show()


## Highest prediction

In [ ]:
print(f'Tanimoto similarity: {golden_truth[target_index, highest_prediction_index]}')

In [ ]:
#print(f'The normalized MCES prediction of SIMBA for the highest truth is {similarities1[target_index, highest_prediction_index]}')

In [ ]:
print(f'The MCES prediction of SIMBA for the highest truth is {40-(40*similarities1[target_index, highest_prediction_index])}')

In [ ]:
print(f'The normalized ed prediction of SIMBA for the highest prediction is {softmax(similarities2[target_index, highest_prediction_index])}')

In [ ]:
print(f'intensities: {np.array(all_spectrums_reference[highest_prediction_index].intensity)/1000}')

In [ ]:
all_spectrums_reference[highest_prediction_index].precursor_mz

In [ ]:
similarities2_softmax[(similarities1>0.95)&(similarities2_norm<1)].shape

## Ground truth

In [ ]:
print(f'Tanimoto similarity: {golden_truth[target_index, best_golden_truth_index]}')

In [ ]:
print(f'The normalized MCES prediction of SIMBA for the golden truth is {similarities1[target_index, best_golden_truth_index]}')

In [ ]:
print(f'The MCES prediction of SIMBA for the golden truth is {40-(40*similarities1[target_index, best_golden_truth_index])}')

In [ ]:
print(f'The normalized ed prediction of SIMBA for the golden truth is {softmax(similarities2[target_index, best_golden_truth_index])}')

In [ ]:
## the best golden truth is ranked

min_ranking=np.inf
for b in best_golden_truth_list:
    ranking_found = np.where(np.argsort(similarities1[target_index])[::-1] == b)[0]
    print(ranking_found)
    min_ranking = ranking_found if ranking_found< min_ranking else min_ranking
print(f'Ranking found for specific ground truth: {min_ranking}')

In [ ]:
## what is the highest prediction?

In [ ]:
all_spectrums_reference[highest_prediction_index].mz

In [ ]:
import spectrum_utils.plot as sup
sup.mirror(target_query, all_spectrums_reference[highest_prediction_index], )

In [ ]:
golden_truth[target_index, highest_prediction_index]

In [ ]:
print(f'The normalized ed prediction of SIMBA for the highest mces  is {softmax(similarities2[target_index, highest_prediction_index])}')

In [ ]:
len(all_spectrums_reference[highest_prediction_index].mz)

In [ ]:
sup.mirror(target_query, all_spectrums_reference[highest_prediction_index], )

In [ ]:
more_informative_spectrums[highest_prediction_index]

In [ ]:
best_golden_truth_list

In [ ]:
for b in best_golden_truth_list:
    plt.figure()
    sup.mirror(target_query, all_spectrums_reference[b], )

## There are spectrums that have generate more high similarity predictions?

In [ ]:
similarities1.shape

In [ ]:
#number_similar_spectra=np.sum(golden_truth>0.9, axis=0)
number_similar_spectra=np.sum(similarities1>0.95, axis=0)
number_similar_spectra = number_similar_spectra>=5

In [ ]:
np.sum(similarities1>0.9)/(similarities1.shape[0]*similarities1.shape[1])

In [ ]:
similarities1.shape[0]*similarities1.shape[1]

In [ ]:
number_similar_spectra

In [ ]:
spectrums_puta= [all_spectrums_reference_processed[i] for i in range(0, len(all_spectrums_reference_processed)) if number_similar_spectra[i]]
len_z_puta = [len(s.mz) for s in spectrums_puta]
len_z_ref = [len(s.mz) for s in all_spectrums_reference_processed]

In [ ]:
_=plt.hist(len_z_puta, bins=30, label='prone to more false positives', alpha=0.5, density=True)
_=plt.hist(len_z_ref, bins=30, label='ref',alpha=0.5,density=True)
plt.legend()

In [ ]:
## plot random similarity 1
similarities1_high= np.where(similarities1>0.98)
similarities1_high

In [ ]:
index_random=np.random.randint(0,len(similarities1_high[0]))


s1 = all_spectrums_janssen[similarities1_high[0][index_random]]
s1_copy= copy.deepcopy(s1)
s1_copy = pp.preprocess_spectrum(
            s1_copy,
            fragment_tol_mass=10,
            fragment_tol_mode="ppm",
            min_intensity=0.01,
            max_num_peaks=40,
            scale_intensity='root',
        )

print(f'index janssen: {similarities1_high[0][index_random]}')
s2 = all_spectrums_reference_processed[similarities1_high[1][index_random]]
print(f'Number of peaks s1: {len(s1_copy.mz)}, Number of peaks s2: {len(s2.mz)}')
sup.mirror(s1_copy, s2, )